In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext
import pandas as pd
import numpy as np
import csv

spark = SparkSession.builder.appName("project").getOrCreate()

In [2]:
'''
import  pyspark.sql.functions as psf
from pyspark.sql.types import DoubleType
cr_udf = psf.udf(lambda x,y:float((x-y) / x), DoubleType())
'''

'\nimport  pyspark.sql.functions as psf\nfrom pyspark.sql.types import DoubleType\ncr_udf = psf.udf(lambda x,y:float((x-y) / x), DoubleType())\n'

In [27]:
with open("/home/hz2558/Median.csv") as csvfile:
    csvReader = csv.reader(csvfile)
    Data = list(csvReader)
title = Data[0]
Data = Data[1:]
for i in range(len(Data)):
    Data[i] = list((map(eval, Data[i])))
data = np.array(Data)
zipcode = data[:, 0]
data = np.delete(data, 0, axis = 1)

In [28]:
data

array([[ 199,  199,  200, ..., 1345, 1340, 1338],
       [ 155,  156,  156, ...,  477,  477,  479],
       [ 353,  355,  356, ..., 1639, 1622, 1612],
       ...,
       [  38,   39,   39, ...,   85,   85,   85],
       [ 124,  124,  125, ...,  368,  374,  380],
       [  73,   73,   73, ...,  151,  153,  155]])

In [29]:
interval = 6
i = 0
res = np.array([[0.0]] * len(data))
while i < 270 - interval:
    res = np.insert(res, len(res[0]), (data[:,i+interval] - data[:,i]) / data[:,i], axis = 1)
    i = i + 1
res = np.delete(res, 0, axis = 1) * 100

In [30]:
D = pd.DataFrame(res)
D

,0,1,2,3,4,5,6,7,8,9,...,254,255,256,257,258,259,260,261,262,263
0,0.502513,1.005025,1.000000,1.500000,2.512563,2.500000,3.000000,2.487562,2.475248,1.970443,...,0.833965,0.453172,0.527506,0.225056,0.074627,0.896861,1.578947,1.127820,0.449775,0.149701
1,1.935484,1.282051,1.923077,1.273885,1.910828,1.265823,1.898734,1.898734,1.886792,1.886792,...,0.213220,0.854701,1.713062,2.141328,1.923077,1.489362,1.489362,1.059322,0.421053,0.419287
2,1.983003,1.690141,1.966292,2.240896,2.513966,2.785515,3.055556,3.324100,3.581267,3.835616,...,-2.998847,-3.705848,-3.902155,-3.747073,-3.225806,-2.237927,-1.486326,-1.443175,-1.696970,-1.946472
3,-1.492537,-4.411765,-4.411765,-4.477612,-4.477612,-5.970149,-4.545455,-3.076923,-1.538462,1.562500,...,0.909091,0.909091,0.909091,1.818182,1.818182,1.818182,0.900901,1.801802,1.801802,0.892857
4,1.522843,1.010101,0.000000,0.000000,-0.500000,-0.500000,-1.000000,-1.500000,-2.000000,-1.500000,...,0.984252,1.176471,1.960784,2.352941,2.152642,1.949318,1.949318,1.550388,0.961538,0.766284
5,1.851852,1.851852,0.000000,0.000000,1.851852,1.851852,0.000000,0.000000,1.818182,1.851852,...,1.176471,1.176471,2.352941,1.162791,1.162791,1.162791,2.325581,2.325581,2.298851,2.298851
6,2.040816,2.040816,2.040816,2.040816,2.040816,-2.000000,-2.000000,-2.000000,0.000000,0.000000,...,3.488372,3.488372,4.651163,4.597701,4.545455,3.409091,2.247191,3.370787,2.222222,1.098901
7,2.040816,0.000000,2.040816,0.000000,0.000000,-4.000000,-4.000000,-4.000000,-4.000000,-2.040816,...,1.204819,1.204819,2.409639,2.409639,1.190476,2.380952,2.380952,2.380952,2.352941,3.529412
8,0.751880,0.374532,1.123596,1.872659,2.247191,2.621723,2.611940,2.985075,2.222222,1.470588,...,-0.140056,-0.766017,-0.556715,-0.691563,-0.276434,0.486449,1.051893,0.701754,0.349895,-0.069638
9,-1.149425,-1.136364,-1.136364,-1.136364,0.000000,0.000000,1.162791,1.149425,1.149425,1.149425,...,3.252033,2.811245,3.600000,3.984064,3.162055,2.755906,3.543307,3.125000,2.702703,2.681992


In [31]:
D.to_csv("new_median.csv", index = False, sep=',')

df = spark.read.csv("/home/hz2558/new_median.csv", inferSchema=True, encoding='utf-8')

In [32]:
df.show(3)

+------------------+------------------+------------------+------------------+-----------------+------------------+----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+------------------+-----------------+-----------------+----------------+------------------+-----------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+--------------

In [33]:
from pyspark.ml.feature import VectorAssembler
# Load training data
vecAss = VectorAssembler(inputCols=df.columns[1:-1], outputCol='features')
df_va = vecAss.transform(df)
training = df_va.select(df_va._c263, df_va.features).selectExpr("_c263 as label", "features as features")
training.show(5) 

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|              263.0|[1.0,2.0,3.0,4.0,...|
|0.14970059880239522|[1.00502512562814...|
|0.41928721174004197|[1.28205128205128...|
|-1.9464720194647203|[1.69014084507042...|
| 0.8928571428571428|[-4.4117647058823...|
+-------------------+--------------------+
only showing top 5 rows



In [34]:
from pyspark.sql import SQLContext
s = training.toPandas()
s = s[1:]
training = spark.createDataFrame(s)
training.show(5)

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|0.14970059880239522|[1.00502512562814...|
|0.41928721174004197|[1.28205128205128...|
|-1.9464720194647203|[1.69014084507042...|
| 0.8928571428571428|[-4.4117647058823...|
| 0.7662835249042145|[1.01010101010101...|
+-------------------+--------------------+
only showing top 5 rows



In [35]:
from pyspark.ml.regression import GeneralizedLinearRegression

glr = GeneralizedLinearRegression(family="gaussian", link="identity", maxIter=10, regParam=0.3)

# Fit the model
glrModel = glr.fit(training)
'''
# Print the coefficients and intercept for generalized linear regression model
print("Coefficients: " + str(glrModel.coefficients))
print("Intercept: " + str(glrModel.intercept))

# Summarize the model over the training set and print out some metrics
summary = glrModel.summary
print("Coefficient Standard Errors: " + str(summary.coefficientStandardErrors))
print("T Values: " + str(summary.tValues))
print("P Values: " + str(summary.pValues))
print("Dispersion: " + str(summary.dispersion))
print("Null Deviance: " + str(summary.nullDeviance))
print("Residual Degree Of Freedom Null: " + str(summary.residualDegreeOfFreedomNull))
print("Deviance: " + str(summary.deviance))
print("Residual Degree Of Freedom: " + str(summary.residualDegreeOfFreedom))
print("AIC: " + str(summary.aic))
print("Deviance Residuals: ")
summary.residuals().show()
'''

'\n# Print the coefficients and intercept for generalized linear regression model\nprint("Coefficients: " + str(glrModel.coefficients))\nprint("Intercept: " + str(glrModel.intercept))\n\n# Summarize the model over the training set and print out some metrics\nsummary = glrModel.summary\nprint("Coefficient Standard Errors: " + str(summary.coefficientStandardErrors))\nprint("T Values: " + str(summary.tValues))\nprint("P Values: " + str(summary.pValues))\nprint("Dispersion: " + str(summary.dispersion))\nprint("Null Deviance: " + str(summary.nullDeviance))\nprint("Residual Degree Of Freedom Null: " + str(summary.residualDegreeOfFreedomNull))\nprint("Deviance: " + str(summary.deviance))\nprint("Residual Degree Of Freedom: " + str(summary.residualDegreeOfFreedom))\nprint("AIC: " + str(summary.aic))\nprint("Deviance Residuals: ")\nsummary.residuals().show()\n'

In [36]:
result = glrModel.transform(training)
result.show(5)

+-------------------+--------------------+-------------------+
|              label|            features|         prediction|
+-------------------+--------------------+-------------------+
|0.14970059880239522|[1.00502512562814...|0.37408500700985986|
|0.41928721174004197|[1.28205128205128...| 0.2601180664077356|
|-1.9464720194647203|[1.69014084507042...| -1.203710744190632|
| 0.8928571428571428|[-4.4117647058823...| 1.7543276365154041|
| 0.7662835249042145|[1.01010101010101...| 0.5604403047761547|
+-------------------+--------------------+-------------------+
only showing top 5 rows



In [37]:
from pyspark.ml.evaluation import RegressionEvaluator
glr_evaluator = RegressionEvaluator(labelCol='label', metricName="rmse", predictionCol='prediction')
rmse = glr_evaluator.evaluate(result)
print("RMSE: %f" % rmse)

RMSE: 0.989416


In [38]:
D

,0,1,2,3,4,5,6,7,8,9,...,254,255,256,257,258,259,260,261,262,263
0,0.502513,1.005025,1.000000,1.500000,2.512563,2.500000,3.000000,2.487562,2.475248,1.970443,...,0.833965,0.453172,0.527506,0.225056,0.074627,0.896861,1.578947,1.127820,0.449775,0.149701
1,1.935484,1.282051,1.923077,1.273885,1.910828,1.265823,1.898734,1.898734,1.886792,1.886792,...,0.213220,0.854701,1.713062,2.141328,1.923077,1.489362,1.489362,1.059322,0.421053,0.419287
2,1.983003,1.690141,1.966292,2.240896,2.513966,2.785515,3.055556,3.324100,3.581267,3.835616,...,-2.998847,-3.705848,-3.902155,-3.747073,-3.225806,-2.237927,-1.486326,-1.443175,-1.696970,-1.946472
3,-1.492537,-4.411765,-4.411765,-4.477612,-4.477612,-5.970149,-4.545455,-3.076923,-1.538462,1.562500,...,0.909091,0.909091,0.909091,1.818182,1.818182,1.818182,0.900901,1.801802,1.801802,0.892857
4,1.522843,1.010101,0.000000,0.000000,-0.500000,-0.500000,-1.000000,-1.500000,-2.000000,-1.500000,...,0.984252,1.176471,1.960784,2.352941,2.152642,1.949318,1.949318,1.550388,0.961538,0.766284
5,1.851852,1.851852,0.000000,0.000000,1.851852,1.851852,0.000000,0.000000,1.818182,1.851852,...,1.176471,1.176471,2.352941,1.162791,1.162791,1.162791,2.325581,2.325581,2.298851,2.298851
6,2.040816,2.040816,2.040816,2.040816,2.040816,-2.000000,-2.000000,-2.000000,0.000000,0.000000,...,3.488372,3.488372,4.651163,4.597701,4.545455,3.409091,2.247191,3.370787,2.222222,1.098901
7,2.040816,0.000000,2.040816,0.000000,0.000000,-4.000000,-4.000000,-4.000000,-4.000000,-2.040816,...,1.204819,1.204819,2.409639,2.409639,1.190476,2.380952,2.380952,2.380952,2.352941,3.529412
8,0.751880,0.374532,1.123596,1.872659,2.247191,2.621723,2.611940,2.985075,2.222222,1.470588,...,-0.140056,-0.766017,-0.556715,-0.691563,-0.276434,0.486449,1.051893,0.701754,0.349895,-0.069638
9,-1.149425,-1.136364,-1.136364,-1.136364,0.000000,0.000000,1.162791,1.149425,1.149425,1.149425,...,3.252033,2.811245,3.600000,3.984064,3.162055,2.755906,3.543307,3.125000,2.702703,2.681992


In [39]:
zipcode = list(zipcode)
for i in range(len(zipcode)):
    zipcode[i] = str(zipcode[i])
    if len(zipcode[i]) == 4:
        zipcode[i] = "0" + zipcode[i]
D.insert(0, "zipcode", zipcode)


In [43]:
result = result.toPandas()
D["prediction"] = result['prediction']
D

,zipcode,0,1,2,3,4,5,6,7,8,...,255,256,257,258,259,260,261,262,263,prediction
0,10025,0.502513,1.005025,1.000000,1.500000,2.512563,2.500000,3.000000,2.487562,2.475248,...,0.453172,0.527506,0.225056,0.074627,0.896861,1.578947,1.127820,0.449775,0.149701,0.374085
1,60657,1.935484,1.282051,1.923077,1.273885,1.910828,1.265823,1.898734,1.898734,1.886792,...,0.854701,1.713062,2.141328,1.923077,1.489362,1.489362,1.059322,0.421053,0.419287,0.260118
2,10023,1.983003,1.690141,1.966292,2.240896,2.513966,2.785515,3.055556,3.324100,3.581267,...,-3.705848,-3.902155,-3.747073,-3.225806,-2.237927,-1.486326,-1.443175,-1.696970,-1.946472,-1.203711
3,77494,-1.492537,-4.411765,-4.411765,-4.477612,-4.477612,-5.970149,-4.545455,-3.076923,-1.538462,...,0.909091,0.909091,1.818182,1.818182,1.818182,0.900901,1.801802,1.801802,0.892857,1.754328
4,60614,1.522843,1.010101,0.000000,0.000000,-0.500000,-0.500000,-1.000000,-1.500000,-2.000000,...,1.176471,1.960784,2.352941,2.152642,1.949318,1.949318,1.550388,0.961538,0.766284,0.560440
5,79936,1.851852,1.851852,0.000000,0.000000,1.851852,1.851852,0.000000,0.000000,1.818182,...,1.176471,2.352941,1.162791,1.162791,1.162791,2.325581,2.325581,2.298851,2.298851,3.024820
6,77449,2.040816,2.040816,2.040816,2.040816,2.040816,-2.000000,-2.000000,-2.000000,0.000000,...,3.488372,4.651163,4.597701,4.545455,3.409091,2.247191,3.370787,2.222222,1.098901,2.021342
7,77084,2.040816,0.000000,2.040816,0.000000,0.000000,-4.000000,-4.000000,-4.000000,-4.000000,...,1.204819,2.409639,2.409639,1.190476,2.380952,2.380952,2.380952,2.352941,3.529412,2.281971
8,10002,0.751880,0.374532,1.123596,1.872659,2.247191,2.621723,2.611940,2.985075,2.222222,...,-0.766017,-0.556715,-0.691563,-0.276434,0.486449,1.051893,0.701754,0.349895,-0.069638,-0.047278
9,10467,-1.149425,-1.136364,-1.136364,-1.136364,0.000000,0.000000,1.162791,1.149425,1.149425,...,2.811245,3.600000,3.984064,3.162055,2.755906,3.543307,3.125000,2.702703,2.681992,2.382822


In [44]:
new_col = ["zipcode", "prediction"]
a = pd.DataFrame(D,columns = new_col)
a

,zipcode,prediction
0,10025,0.374085
1,60657,0.260118
2,10023,-1.203711
3,77494,1.754328
4,60614,0.560440
5,79936,3.024820
6,77449,2.021342
7,77084,2.281971
8,10002,-0.047278
9,10467,2.382822


In [45]:
a.to_csv("new_Median.csv", index=False, sep=",")